# Build a simulation using the Python API


Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
my_sim.describe_state()

{'uuid': '8d5cbb1b-aa9b-4f66-8b23-80d47755df69',
 'network': {'uuid': 'd3569bc4-eeed-40b1-9c3c-0fe80b9bb11c',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': '4d4024ae-5948-4f07-aed9-d2315891cddc', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.base import Node


In [4]:
my_pc = Node(hostname="primaite_pc",)
my_sim.network.nodes[my_pc.uuid] = my_pc
my_sim.describe_state()

{'uuid': '8d5cbb1b-aa9b-4f66-8b23-80d47755df69',
 'network': {'uuid': 'd3569bc4-eeed-40b1-9c3c-0fe80b9bb11c',
  'nodes': {'5f596f4f-4d34-4d1c-9688-9a105e489444': {'uuid': '5f596f4f-4d34-4d1c-9688-9a105e489444',
    'hostname': 'primaite_pc',
    'operating_state': 0,
    'NICs': {},
    'file_system': {'uuid': 'dc1e7032-7dba-44d5-aedb-5da75ab1eccc',
     'folders': {}},
    'applications': {},
    'services': {},
    'process': {}}},
  'links': {}},
 'domain': {'uuid': '4d4024ae-5948-4f07-aed9-d2315891cddc', 'accounts': {}}}